# Exploretory Data Analysis

### To-Do
- Zusammenhang zwischen 5 Sterne und Review Count
- Bei jedem Review ein von jeder Sorte hinzufügen für fairen durchschnitt
- Daten auf Karte plotten
- Durchschnitte Median Mode berechnen für verschiedene Orte
- Preis durch Rating teilen teuer ==  besser bewertet?
- Art der Restaurant italienisch usw. (auch nach Region)
- Art mit Bewertung vergleichen
- Art mit Preis


In [ ]:
df = pd.read_csv(r"E:\Projekte\Yelp_Analyser\vancouver_yelp_stores.csv")

df.columns

df.describe()

df["stars"].value_counts()
df["stars"].mean()
df["stars"].median()
df["stars"].mode()
df["stars"].isna().sum()


df["review_count"].value_counts()
df["review_count"].mean()
df["review_count"].median()
df["review_count"].mode()
df["review_count"].isna().sum()

df["review_count"].where(df["stars"] == 5.0,).groupby(df["review_count"]).transform('max')